In [1]:
search()

[1] ".GlobalEnv"        "jupyter:irkernel"  "package:stats"    
 [4] "package:graphics"  "package:grDevices" "package:utils"    
 [7] "package:datasets"  "package:methods"   "Autoloads"        
[10] "package:base"

In [2]:
library("readxl")

In [3]:
dataframing <- function(path){
    names <- excel_sheets(path)
    if (length(excel_sheets(path)) == 1){
        df <- data.frame(read_excel(path))
        colnames(df)<-c("Date", names[1])
        return (df)
    } else {
        df<-data.frame(read_excel(path, sheet = 1))
        colnames(df)<-c("Date", names[1])
        for (i in 2:length(excel_sheets(path))){
            tempdf<- read_excel(path, sheet = i)
            colnames(tempdf)<- c("Date", names[i])
            df <- merge(df, tempdf, by = "Date") 
        }
        return (df)
    }
}

In [4]:
path <- "../data/data.xlsx"

In [5]:
df <- dataframing(path)

In [6]:
df

Date,10Y,7Y,5Y,3Y,2Y,1Y,6M,3M
2018-02-16,2.876,2.805,2.630,2.385,2.192,1.975,1.825,1.594
2018-02-23,2.868,2.791,2.619,2.388,2.240,2.001,1.852,1.641
2018-03-02,2.865,2.788,2.629,2.397,2.243,2.044,1.841,1.644
2018-03-09,2.895,2.811,2.651,2.432,2.260,2.023,1.874,1.664
2018-03-16,2.845,2.778,2.643,2.444,2.293,2.054,1.945,1.773
2018-03-23,2.814,2.740,2.600,2.404,2.256,2.017,1.902,1.720
2018-03-30,2.740,2.685,2.563,2.384,2.268,2.090,1.918,1.706
2018-04-06,2.774,2.708,2.587,2.401,2.268,2.049,1.896,1.712
2018-04-13,2.828,2.778,2.673,2.502,2.359,2.095,1.956,1.752
2018-04-20,2.961,2.914,2.801,2.618,2.459,2.195,2.003,1.805
